In [ ]:
#for generating transactions:
# you need : 1- moments : for each moment you will generate one transaction ( if you want to change the nr of transactions, you need to edit moments)
# bank_accounts_ids : to choose one person?  ( or choose from people)
#list of prev categories for the chosen person
# categories_weights to also include weights for categories in that moment
# choose a category
#decide amount
#choose a company ( or a person if  it is a tekkie)
#excute




#this will not include salaries and basic necessities

#2nd step: go throgh moments, filter on first day of the month , keep one moment for each day
# now, go for each day, for each day, go for all people
#for each person : pay all expenses


In [593]:
import pandas as pd
import numpy as np
import string
import random 
import requests
from datetime import datetime
import json
import dask.dataframe as dd


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
 
countries = pd.read_excel('countries.xlsx')
companies= pd.read_excel( 'companies.xlsx')  
people  = pd.read_excel( 'people.xlsx' )   

 
# banks=  pd.read_excel( 'banks.xlsx' )  


bank_accounts_ids  = pd.read_excel( 'bank_accounts_ids.xlsx' )   
transactions=  pd.read_excel( 'transactions.xlsx' ) 
categories_mean_costs_per_country =pd.read_excel( 'categories_mean_costs_per_country.xlsx' )     

print('bank_accounts_ids  transactions  companies  countries people categories_mean_costs_per_country loaded') 


 
cols_to_keep = ['datetime','category','final_weight']
categories_weights  = pd.read_csv( 'categories_weights_2020_2025.csv' , usecols=cols_to_keep )   
print('categories_weights_2020_2025 loaded') 
    
 
moments =      pd.read_excel( 'moments.xlsx' ) 
moments['moment'] = pd.to_datetime(moments['moment'])

print('moments loaded') 

categories_weights_reference = pd.read_excel( 'categories_weights_reference.xlsx' ) 

# Make sure datetime columns are in datetime format

categories_weights['datetime'] = pd.to_datetime(categories_weights['datetime'])

print ('done')

  

bank_accounts_ids  transactions  companies  countries people categories_mean_costs_per_country loaded
categories_weights_2020_2025 loaded
moments loaded
done


In [594]:
# moments =      pd.read_excel( 'moments.xlsx' ) 
moments['moment'] = pd.to_datetime(moments['moment'])
len(moments)

53957

In [595]:
moments.head()

,moment,type
0,2024-01-01 00:00:14,other
1,2024-01-01 00:01:32,other
2,2024-01-01 00:08:27,other
3,2024-01-01 00:10:47,other
4,2024-01-01 00:12:18,other


In [484]:
len(categories_weights)

25382994

In [649]:
 
companies= pd.read_excel( 'companies.xlsx')  
people  = pd.read_excel( 'people.xlsx' )   

 
# banks=  pd.read_excel( 'banks.xlsx' )  


bank_accounts_ids  = pd.read_excel( 'bank_accounts_ids.xlsx' )   

In [596]:
len(categories_mean_costs_per_country)

14742

In [597]:
len(people)

3000

In [492]:
# categories_weights  = pd.read_csv( 'categories_weights_2020_2025.csv' , usecols=cols_to_keep )   


In [494]:
# categories_weights['datetime'] = pd.to_datetime(categories_weights['datetime'])
# print('done')

done


In [175]:
# # just merging two dfs:
# bank_accounts_ids1  = pd.read_excel( 'bank_accounts_ids1.xlsx' )   
# bank_accounts_ids = pd.concat([bank_accounts_ids, bank_accounts_ids1], ignore_index=True)


In [598]:
#functions:


#return a unique idn, you can mention where to search 
#you can decide the nr_digits and nr_letters in the idn
def unique_idn(nr_digits=10, nr_letters=0, search_list=[]):
#         idn= "".join(list (map(str, list( np.random.randint(0,10,nr_digits)))))
        letters =list ( np.random.choice(list (string.ascii_letters), size=nr_letters, replace=True))
        digits = list (np.random.choice(list( string.digits), size=nr_digits  ))
        idn=letters + digits


        random.shuffle(idn)
        idn="".join (idn) 
        
    #if exists: return the function again
        if idn in search_list :
           #print('not unique')
           return unique_idn(nr_digits,search_list)
        return idn
unique_idn(nr_digits=4,nr_letters=3)




'1064OBl'

In [664]:
#takes a base and target currencies and a date, return the exchange rate on that date
import requests
from datetime import datetime
def get_exchange_rate(base_currency='EUR', target_currency='EUR' , date='2024-02-15' )  :
    """
    Fetches the historical exchange rate between two currencies on a given date.

    :param base_currency: The currency code to convert from (e.g., 'USD').
    :param target_currency: The currency code to convert to (e.g., 'EUR').
    :param date: The date for the exchange rate in 'YYYY-MM-DD' format.
    :return: The exchange rate on the specified date.
    """
    if base_currency==target_currency:
        return 1
    try:
        # Ensure the date is in the correct format
        datetime.strptime(date, '%Y-%m-%d')
        url = f"https://api.frankfurter.app/{date}"
        params = {
            'from': base_currency,
            'to': target_currency
        }
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        return data['rates'][target_currency]
    except ValueError:
        print("Error: Incorrect date format. Please use 'YYYY-MM-DD'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching exchange rate: {e}")
    except KeyError:
        print("Error: Invalid currency code or data not available for the specified date.")
    return None

get_exchange_rate(base_currency='USD', target_currency='EUR' , date='2024-02-15' )

0.93084

In [658]:
# transactions=  pd.read_excel( 'transactions.xlsx' )  


#makes a transaction from A to B, write the details in transactions df and updates balance in bank_accounts_ids df
#monthly indicates whether this transaction is related to the monthly expenses / salary
def pay( sender_id = '',   receiver_id= '', amount_from_sender=0, description='', time= '',date='',
        category='',high_level_category='', sender_name='', receiver_name='', monthly='no'):
    
    global transactions
    global bank_accounts_ids
    
    transaction_id= unique_idn(nr_digits=6, nr_letters=6, search_list=transactions.transaction_id.tolist())
    #first adjust the amount according to exchange rate:
    #<<to do : exchange rate funcion: takes two currencies and return the exchange rate>>
    
#     sender_id = int (sender_id) 
#     receiver_id = int ( receiver_id)
    sender_currency =bank_accounts_ids.query("idn==@sender_id")['currency'].values[0]
    receiver_currency= bank_accounts_ids.loc[bank_accounts_ids["idn"] == receiver_id, "currency"].values[0]

    exchange_rate=get_exchange_rate(sender_currency,receiver_currency,date )
    if exchange_rate is None :
        exchange_rate = 1
    
    amount_to_receiver=exchange_rate * amount_from_sender
    #then get the balances after and before:
    sender_balance_before = bank_accounts_ids.loc[bank_accounts_ids["idn"] == sender_id, "balance"].values[0]
    sender_balance_after=  sender_balance_before - amount_from_sender

    receiver_balance_before= bank_accounts_ids.loc[bank_accounts_ids["idn"] == receiver_id, "balance"].values[0]
    receiver_balance_after= receiver_balance_before + amount_to_receiver 
    # print (  receiver_balance_before,receiver_balance_after )
    
     
    
    


    transaction_data = {
    "transaction_id":transaction_id,
    "time":  time,
     "date": date,
    "sender_id": sender_id,
      "sender_name":sender_name,  
    "sender_iban":  bank_accounts_ids.loc[bank_accounts_ids["idn"] == sender_id, "iban"].values[0],
    "sender_country":  bank_accounts_ids.loc[bank_accounts_ids["idn"] == sender_id, "country"].values[0],
    "sender_currency":  sender_currency,
    "sender_balance_before": sender_balance_before,
    "sender_balance_after": sender_balance_after,
    "receiver_id": receiver_id,
       "receiver_name":receiver_name,
    "receiver_iban":  bank_accounts_ids.loc[bank_accounts_ids["idn"] == receiver_id, "iban"].values[0],
    "receiver_country":  bank_accounts_ids.loc[bank_accounts_ids["idn"] == receiver_id, "country"].values[0],
    "receiver_currency":  receiver_currency,
    "receiver_balance_before": receiver_balance_before,
    "receiver_balance_after":receiver_balance_after,
    "exchange_rate": exchange_rate,
    "description": description ,
      'amount_from_sender': amount_from_sender  	,
        'amount_to_receiver': amount_to_receiver , 
'category':category,
        'high_level_category' : high_level_category,
        'monthly':monthly 
    } 

    transaction_data
    transaction_data= pd.DataFrame([transaction_data])
    transaction_data
    
    transactions= pd.concat([transactions, transaction_data], ignore_index=True)

    # #now update the balance for both:
    bank_accounts_ids.loc[bank_accounts_ids["idn"] == sender_id, "balance"] = sender_balance_after
    bank_accounts_ids.loc[bank_accounts_ids["idn"] == receiver_id, "balance"] = receiver_balance_after

    

    
    
#pay(sender_id = '8115971985' , receiver_id= '1302797885',amount_from_sender=100,description ='tekkie',date= '2024-02-15' )    
    
    
# # transaction_data
# transactions  

In [667]:
#function : moment > transaction 
def generate_transaction ( moment):
     

#         categories_weights['datetime'] = pd.to_datetime(categories_weights['datetime'])

        # for moment in moments:

        #     choose someone
        person = people.sample(n=1)  
        idn = person['idn'].values[0]
        country = person['country'].values[0]
        sender_name = f"{ person['first_name'].values[0] } { person['last_name'].values[0] }"

        #     filter categories_wieghts on that time
        target_hour = moment.floor('H')
        #this will get us all categories and their weights in that hour
        filtered_categories_weights = categories_weights[ categories_weights['datetime'].dt.floor('H') == target_hour]
        # print ( filtered_categories_weights)
        #     get a list of categories and their weights

        #     get  the prev categories for that person, append it to the filterd categories
        #for monthly =no, we want only categories that are not the monthly expenses

        list_prev_categories =  transactions.query("sender_id==@idn and monthly =='no'").category.tolist()
        prev_categories_df = pd.DataFrame({
        'category': list_prev_categories,
        'final_weight': [5] * len(list_prev_categories)}) #here this is just an assumption
#         print(prev_categories_df)
        #append the list_prev_categories
        filtered_categories_weights = pd.concat([filtered_categories_weights, prev_categories_df], ignore_index=True)

        #     choose a category

        list_categories = filtered_categories_weights['category'].tolist()
        weights = filtered_categories_weights['final_weight'].tolist()


        chosen_category = random.choices(list_categories, weights=weights, k=1)[0]
#         print (chosen_category)
        #     get the mean cost for it
        if len (categories_mean_costs_per_country.query( "country==@country and category==@chosen_category")) >0:

         mean_cost= categories_mean_costs_per_country.query( "country==@country and category==@chosen_category").sample(n=1)['mean_cost'].values[0]
        elif  len (categories_mean_costs_per_country.query( "country==@country  ")) : #get the cost of sth else ( very random)
           mean_cost= categories_mean_costs_per_country.query( "country==@country ")['mean_cost'].values[0]
        else : 
            mean_cost=50 # just an assumption
        # print (chosen_category, country, mean_cost)    

        mean_cost=int(mean_cost)
        amount =  round  (np.random.normal(loc=mean_cost, scale=mean_cost*0.1) ,2)
        #     choose a destination: first : what is the high_level_category?
        high_level_category = categories_basic_expanded.query("transaction_categories==@chosen_category")['high_level_category'].values[0]

        if chosen_category == 'Tikkie' :
            destination = people.query("country==@country").sample(n=1)
            receiver_name = f"{ destination['first_name'].values[0] } { destination['last_name'].values[0] }"


        else:


             # get a list of companies in that country with this 


            possible_high_level_categories = categories_basic_expanded.query("transaction_categories==@chosen_category")['high_level_category'].tolist()

            #now based on the chosen categoy, choose one company in that country, which has the high_level_catgory
            # of one of the possible high_level_categories
            if len(companies.query( "high_level_category in @possible_high_level_categories and country == @country")) > 0:
                destination=  companies.query( "high_level_category in @possible_high_level_categories and country == @country").sample(n=1)

            else:
                #so if there's no company for that category in that country, choose from all countries
                destination=  companies.query( "high_level_category in @possible_high_level_categories").sample(n=1)
            receiver_name =destination['company'].values[0]    


        # now you have everything, generate a transaction
        destination
        sender_id = idn 
        receiver_id =  destination['idn'].values[0]
        amount_from_sender = amount
        time = moment
        category = chosen_category
        date = str (moment.date())
        sender_name = sender_name
        receiver_name = receiver_name
        high_level_category = high_level_category

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
            description ='',time= time , date=date, category= category, high_level_category = high_level_category,
            sender_name= sender_name, receiver_name= receiver_name
           ) 
#         print ('done')
        
        
        
generate_transaction  (moments.iloc[0][0])

In [587]:
categories_mean_costs_per_country.columns

Index(['country', 'high_level_category', 'category', 'mean_cost'], dtype='object')

In [656]:
def pay_expenses(moment, people) :
    
    # person > pay all expenses:
    for index, row in people.iterrows():
        person = pd.DataFrame([row])    
    #     moment = moments.iloc[[0]]['moment'][0]

        country= person['country'].values[0]
#         print (country)
         

        # getting salary: 
        receiver_id = person['idn'].values[0]
        receiver_name = f"{ person['first_name'].values[0] } { person['last_name'].values[0] }"

        sender_name =   person['work_company'].values[0]  
        sender_id =     person['id_work_company'].values[0]  

        amount  = int (   person['salary'].values[0]  )
        category = 'salary'
        high_level_category = 'Work'
        description =f"salary for {moment.strftime('%Y-%m')}"
        date = str (moment.date())

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
        description =description ,time= moment , date=date, category= category, high_level_category = high_level_category,
        sender_name= sender_name, receiver_name= receiver_name, monthly ='yes'  )



        ##
        ################paying expenses:###################
        sender_id = person['idn'].values[0]
        sender_name = f"{ person['first_name'].values[0] } { person['last_name'].values[0] }"
        # housing:
        receiver_name =   person['housing_company'].values[0]  
        receiver_id =     person['id_housing_company'].values[0]  
        mean_monthly_rent_costs =int  (person['mean_monthly_rent_costs'].values[0]     )
        amount  = int ( np.random.normal(loc=mean_monthly_rent_costs, scale=mean_monthly_rent_costs *0.3))
        category = 'Rent'
        description =f"paying rent for {moment.strftime('%Y-%m')}"
        high_level_category = 'Housing'
        date = str (moment.date())

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
        description =description ,time= moment , date=date, category= category, high_level_category = high_level_category,
        sender_name= sender_name, receiver_name= receiver_name , monthly ='yes' )
    #     print('done')

    # energy:

        receiver_name =   person['energy_company'].values[0]  
        receiver_id =     person['id_energy_company'].values[0]  
        mean_monthly_energy_costs =int  (person['mean_monthly_energy_costs'].values[0]     )
        amount  = int ( np.random.normal(loc=mean_monthly_energy_costs, scale=mean_monthly_energy_costs *0.3))
        category = 'Electricity'
        description =f"paying electricity and gas for {moment.strftime('%Y-%m')}"
        high_level_category = 'Energy'
        date = str (moment.date())

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
        description =description ,time= moment , date=date, category= category, high_level_category = high_level_category,
        sender_name= sender_name, receiver_name= receiver_name, monthly ='yes'  )
    #     print('energy done')

    #health insurance:

        receiver_name =   person['health_ins_company'].values[0]  
        receiver_id =     person['id_health_ins_company'].values[0]  
        monthly_health_ins_costs =int  (person['monthly_health_ins_costs'].values[0]     )
        amount  = int ( np.random.normal(loc=monthly_health_ins_costs, scale=mean_monthly_energy_costs *0.2))
        category = 'Health Plans'
        description =f"paying health insurance for {moment.strftime('%Y-%m')}"
        high_level_category = 'Health Insurance'
        date = str (moment.date())

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
        description =description ,time= moment , date=date, category= category, high_level_category = high_level_category,
        sender_name= sender_name, receiver_name= receiver_name, monthly ='yes'  )
#         print('Health done')

    #internet:

        receiver_name =   person['internet_company'].values[0]  
        receiver_id =     person['id_internet_company'].values[0]  
        monthly_internet_costs =int  (person['monthly_internet_costs'].values[0]     )
        amount  = int ( np.random.normal(loc=monthly_internet_costs, scale=mean_monthly_energy_costs *0.1))
        category = 'Subscription Services'
        description =f"paying internet costs for {moment.strftime('%Y-%m')}"
        high_level_category = 'Internet'
        date = str (moment.date())

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
        description =description ,time= moment , date=date, category= category, high_level_category = high_level_category,
        sender_name= sender_name, receiver_name= receiver_name, monthly ='yes' )
#         print('internet done')

        groceries_company = companies.query("country == @ country and high_level_category=='Groceries'").sample(n=1)
#         print  (groceries_company)
        receiver_id =    groceries_company['idn'].values[0]
        receiver_name =   groceries_company['company'].values[0]
        
        monthly_groceries_costs =int  (person['monthly_groceries_costs'].values[0]     )
        amount  = int ( np.random.normal(loc=monthly_groceries_costs, scale=monthly_groceries_costs *0.1))
        category = 'Supermarket Items'
        description =f"buying  groceries for {moment.strftime('%Y-%m')}"
        high_level_category = 'Groceries'
        date = str (moment.date())

        pay(sender_id =sender_id , receiver_id=  receiver_id,amount_from_sender=amount,
        description =description ,time= moment , date=date, category= category, high_level_category = high_level_category,
        sender_name= sender_name, receiver_name= receiver_name, monthly ='yes' )


moment = moments.iloc[0][0]
pay_expenses(moment, people.sample(n=55)) 
print('done')

done


In [641]:
people.head(2)

,idn,first_name,last_name,gender,country,country_code,exchange_rate,male_prob,mean_age,age,education_level,education_title,profession,years_experience,working,source_income,salary_eur,salary,initial_balance_eur,initial_balance,hobby,spendingBehaviour,work_company,id_work_company,id_housing_company,id_energy_company,id_health_ins_company,id_internet_company,housing_company,energy_company,health_ins_company,internet_company,mean_monthly_rent_costs,mean_monthly_energy_costs,mean_monthly_internet_costs,mean_monthly_health_ins_costs,monthly_rent_costs,monthly_energy_costs,monthly_internet_costs,monthly_health_ins_costs,bank_name,bank_idn,mean_monthly_groceries_costs,monthly_groceries_costs
0,4850222937,Tereza,Antonín,female,Czech Republic,CZ,25.0,0.492,46.1,68,Postgraduate,Master of Engineering,Electrical Engineer,38,0,social_benefits,1684.8,42120.0,33526,838150.0,Origami,Middle-Class_Value_Seeker,Czech Republic GOV,5641084887,9665268332,9339131678,2031525277,6998138161,Triple L Group,ČEZ,Allianz CZ,O2 Czech Republic,20000,3000,600,4000,22769.37,2918.74,608.57,4624.87,Česká spořitelna,8948097501,7500,6494.91
1,1695240261,Teresa,Tiago,female,Portugal,PT,1.0,0.479,49.2,43,Postgraduate,Master of Business Administration,Business Manager,19,1,work,3150.0,3150.0,118867,118867.0,Photography,Middle-Class_Value_Seeker,Novabase,5682059900,698425747,8741024410,5818235290,1992118347,Mota-Engil,Galp Energia,Zurich Portugal,NOS,800,130,30,180,753.75,115.93,31.22,137.97,Banco BPI,2473225034,290,202.78


In [647]:
sender_id= int ('6998138161') 
bank_accounts_ids.query("idn==@sender_id")

,idn,iban,acc_nr,currency,country,country_code,balance,bank_name,bank_idn
997,6998138161,CZ6027007751105510103,7751105510103,CZK,Czech Republic,CZ,8.000000e+09,UniCredit Bank Czech Republic,1110463937


In [648]:
companies.query("idn=='5641084887'")

,country,company,high_level_category,total_capacity,nr_employees,idn,acc_nr,iban,balance,transaction_categories


In [606]:
bank_accounts_ids.query("idn==4850222937")

,idn,iban,acc_nr,currency,country,country_code,balance,bank_name,bank_idn
4698,4850222937,CZ5550009073019112,9073019112,CZK,Czech Republic,CZ,838150.0,Raiffeisenbank Czech Republic,3464617153


In [666]:
# transactions =  transactions.iloc[0:0] 
 
transactions

,transaction_id,date,time,sender_id,sender_iban,sender_country,sender_currency,sender_balance_before,sender_balance_after,receiver_id,receiver_iban,receiver_country,receiver_currency,receiver_balance_before,receiver_balance_after,exchange_rate,amount_from_sender,amount_to_receiver,description,category,monthly,sender_name,receiver_name,high_level_category


In [448]:
# bank_accounts_ids.query("idn==4850222937")

,idn,iban,acc_nr,currency,country,country_code,balance,bank_name,bank_idn
2559,4850222937,CZ93600371110233800,371110233800,CZK,Czech Republic,CZ,4531410.0,Moneta Money Bank,2508475131


In [578]:
# bank_accounts_ids  = pd.read_excel( 'bank_accounts_ids.xlsx' )   
len(bank_accounts_ids)

5559

In [591]:
#now looping through all moments
transactions =  transactions.iloc[0:0] 
 
for index, row in moments.head(100).iterrows():
    moment = row['moment'] 
    
    if row['type']=='monthly':
        
        pay_expenses(moment, people.head(1)) 
    else:
        generate_transaction(moment)
        
        
        
len ( transactions)      

100

In [668]:
transactions

,transaction_id,date,time,sender_id,sender_iban,sender_country,sender_currency,sender_balance_before,sender_balance_after,receiver_id,receiver_iban,receiver_country,receiver_currency,receiver_balance_before,receiver_balance_after,exchange_rate,amount_from_sender,amount_to_receiver,description,category,monthly,sender_name,receiver_name,high_level_category
0,c7T1vm679ww5,2024-01-01,2024-01-01 00:00:14,8539383188,SE251100246116211903,Sweden,SEK,1989422.65,1989218.42,5150831205,SE91800011101180102004,Sweden,SEK,2720005720.86,2720005925.09,1,204.23,204.23,,Flight Tickets,no,Viktor Emil,Aviation Inc.,Aviation


In [526]:
index

6735

In [657]:
#note: this will export to different  files, you need to merge later


# Number of chunks (5 = 20% each)
n_chunks = 5
# moments_test = moments.iloc[:50]
moments_test = moments 

chunk_size = len(moments_test) // n_chunks
for i in range(n_chunks):
    # Get the chunk
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(moments_test)
    moments_chunk = moments_test.iloc[start:end]
    print (f"i is {i}" )
#     print (start , end)
#     Clear transactions before processing
    transactions = transactions.iloc[0:0]
    
    # Process the current chunk
    for index, row in moments_chunk.iterrows():
        moment = row['moment']
        print(len (transactions), end='\r')
        if row['type'] == 'monthly':
            pay_expenses(moment, people)
        else:
            generate_transaction(moment)
    
    # Save after each chunk
    transactions.to_csv(f"transactions/transaction{i+1}.csv", index=False)


i is 0
i is 1
i is 2
i is 3
i is 4


In [ ]:
# #if you want to generate transactions in one go: 

# # Clear transactions before processing
# transactions = transactions.iloc[0:0]

# # Process the current chunk
# for index, row in moments_chunk.iterrows():
#     moment = row['moment']
#     print(len (transactions), end='\r')
#     if row['type'] == 'monthly':
#         pay_expenses(moment, people)
#     else:
#         generate_transaction(moment)

# # Save after each chunk
# transactions.to_csv(f"transactions.csv", index=False)